# Introduction to Building LLM Agents with Tools and Tracing

<!--- @wandbcode{bedrock-webinar-July-2025} -->

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!pip install weave requests openai
```


In [101]:
# Global Configuration & Setup
import os
import inspect
import json
import weave # Must import weave before litellm for auto-patching
from enum import Enum
from pydantic import BaseModel, Field
from rich.pretty import pprint
from rich.markdown import Markdown
from rich.console import Console
from openai import OpenAI
from exa_py import Exa
from typing import Any, Callable, Dict, List, get_type_hints

Define a model to use, as we are going to use tool calling you need a capable model like `mistral-large`

In [107]:
console = Console()
def md(text): return console.print(Markdown(text))
# Set the model name you want to use globally.

MODEL_SMALL = "Qwen/Qwen3-235B-A22B-Instruct-2507"
MODEL_MEDIUM = "moonshotai/Kimi-K2-Instruct"
MODEL_LARGE = "zai-org/GLM-4.5"

oai_client = OpenAI(
    base_url='https://api.inference.wandb.ai/v1',
    api_key=os.getenv("WANDB_API_KEY"),
    project="milieu/london-workshop-2025")
exa_client = Exa(api_key=os.getenv("EXA_API_KEY"))

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [103]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init('milieu/london-workshop-2025')

/Users/tcapelle/work/deep-research-bot/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

weave: Logged in as Weights & Biases user: capecape.
weave: View Weave data at https://wandb.ai/milieu/london-workshop-2025/weave


## 1. Basic LLM Call with OpenAI SDK

Let's start with a simple call to the LLM using/

![](images/01_trace.png)

In [ ]:
# Define a simple message list (conversation history)
messages = [{"role": "user", "content": "Hello, LLM! How does an AI agent work?"}]

# Make the call
response = resp = oai_client.chat.completions.create(
    model = MODEL_SMALL,
    messages=messages,
)
# Print the response content
assistant_response = response.choices[0].message.content
md(f"LLM Response:\\n{assistant_response}")

# Click on the 🍩 linke below to see the trace in Weave 👇

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed0b-5a45-7aef-a1d0-447b246c8594


LLM Response:\nHello! Great question! 😊 An AI agent is a software system that perceives its environment, makes    
decisions, and takes actions to achieve specific goals. Think of it as a "digital entity" that can act             
autonomously, much like a robot in the physical world—but often operating in digital environments like websites,   
databases, or virtual spaces.                                                                                      

Here’s a breakdown of how an AI agent works:                                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using sensors or data inputs. These could include:              

 • Text input (like your message to me)                                                                            
 • Sensor data (in robotics)                                                                                       
 • Camera images or audio (in voice assistants)                                                                    
 • API data, web pages, or user behavior logs                                                                      

👉 Example: A chatbot reads your message to understand what you're asking.                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     2. Decision-Making (Reasoning & Planning)                                     

Using the input, the agent processes information and decides what to do. This often involves:                      

 • Natural Language Understanding (NLU): To interpret human language.                                              
 • Knowledge Base or Memory: Accessing stored information or past experiences.                                     
 • Reasoning Models: Using logic, rules, or machine learning (like neural networks) to evaluate options.           
 • Goal-Driven Planning: Breaking down a task into steps (e.g., book a flight → search flights → compare prices →  
   confirm booking).                                                                                               

👉 Example: A virtual assistant plans a route by analyzing traffic data and your calendar.                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs an action based on its decision. Actions might include:                                         

 • Sending a message (like this one!)                                                                              
 • Moving a robot arm                                                                                              
 • Updating a database                                                                                             
 • Clicking buttons in a web browser (via automation)                                                              
 • Recommending a product                                                                                          

👉 Example: An AI agent replies to your email or turns on your smart lights.                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        4. Learning (Optional but Powerful)                                        

Many modern AI agents can learn from experience using:                                                             

 • Reinforcement Learning: Learning by trial

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed0c-02ff-730b-b9b7-1b1af36d3e57


Because we imported `weave` and called `weave.init()`, the OpenAI SDK call above was automatically traced. You can open your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [ ]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: List[Dict[str, Any]], **kwargs) -> str:
    "Call a model with the given messages and kwargs."
    response = oai_client.chat.completions.create(
        model=model_name,
        messages=messages,
        **kwargs
    )

    return response.choices[0].message

response = call_model(model_name=MODEL_SMALL, messages=messages)
md(response.content)

Hello! Great question! 😊                                                                                          

An AI agent is a software system that perceives its environment, makes decisions, and takes actions to achieve     
specific goals. Think of it as a "digital being" that can observe, think, and act—often autonomously.              

Here’s a breakdown of how an AI agent works:                                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                      1. Perception (Sensing the Environment)                                      

The agent gathers information from its environment using:                                                          

 • Sensors (in robotics: cameras, microphones, etc.)                                                               
 • Data inputs (in software: user inputs, APIs, databases, text, etc.)                                             

For example:                                                                                                       

 • A chatbot reads your message.                                                                                   
 • A self-driving car uses cameras and lidar to "see" the road.                                                    

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                   2. Processing & Decision-Making (The "Brain")                                   

This is where AI models (like neural networks or rule-based systems) come in. The agent:                           

 • Analyzes the input data.                                                                                        
 • Uses internal models (e.g., machine learning, logic rules) to understand the situation.                         
 • Decides what action to take based on its goals.                                                                 

🧠 Common techniques:                                                                                              

 • Machine Learning (e.g., classifying images, predicting outcomes)                                                
 • Reinforcement Learning (learning optimal actions through trial and reward)                                      
 • Planning Algorithms (e.g., finding the shortest path)                                                           
 • Large Language Models (LLMs) (e.g., generating responses in chatbots)                                           

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                       3. Action (Acting on the Environment)                                       

The agent performs actions to influence its environment:                                                           

 • A robot moves its arm.                                                                                          
 • A recommendation system suggests a movie.                                                                       
 • A chatbot replies to your message.                                                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     4. Feedback Loop (Learning & Improvement)                                     

Many agents learn from experience:                                                                                 

 • They observe the outcome of their actions.                                                                      
 • Adjust behavior to improve performance over time.                                                               

🔁 This creates a cycle: Perceive → Thin

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed0c-9edd-7552-b6bd-b81fb7df3971
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed0e-78af-77f5-b088-193fb7a14dc3
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed11-dae1-75b7-8d69-1abac175470c


![](images/02_nested_trace.png)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

Check the Mistral docs for function calling: https://platform.openai.com/docs/guides/function-calling

![](images/function-calling-diagram-steps.png)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [110]:
@weave.op 
def add_numbers(a: int, b: int) -> int:
    """Adds two numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [ ]:
add_numbers(1, 2)

3

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed0b-fbf3-7e94-a606-957cc45d5c61


weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed0e-7211-7def-adbe-7732bd367cc6
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed10-adab-7da0-bf7c-a1749a1f1eae
weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed11-4857-7895-86e0-ce6c0393338a


In [112]:
# this doesn't work...
call_model(model_name=MODEL_SMALL, messages=messages, tools=[add_numbers])

TypeError: Object of type function is not JSON serializable

> We need to manually create the JSON schema describing this tool in a format that models *Mistral* understand.

In [113]:
# Manually define the tool schema
tool_add_numbers_schema = {
    "type": "function",
    "function": {
        "name": "add_numbers",
        "description": "Adds two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first number."
                },
                "b": {
                    "type": "integer",
                    "description": "The second number."
                }
            },
            "required": ["a", "b"]
        }
    }
}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [115]:
messages = [
    {"role": "system", "content": "You are a helpful assistant use tools to answer questions."},
    {"role": "user", "content": "My lucky numbers are 77 and 11. What is their sum?"}]
response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[tool_add_numbers_schema])
console.print(response)

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed0c-38bf-787d-9698-e5154e7a8e32


ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-b9a99083698b4085a853f8d9748d6ef2',
            function=Function(arguments='{"a": 77, "b": 11}', name='add_numbers'),
            type='function'
        )
    ],
    reasoning_content=None
)

## Manual Tool Call
The LLM's response might contain a request to call our tool (`response.choices[0].message.tool_calls`) or it might respond directly (`response.choices[0].message.content`). If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [ ]:
if response.tool_calls:
    console.print("LLM requested a tool call:")
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args_str = tool_call.function.arguments
        function_args = json.loads(function_args_str)
        console.print(f"  - Tool: {function_name}, Args: {function_args_str}")
        if function_name == "add_numbers":
            tool_result_content = add_numbers(**function_args)

console.print(f"Final Result: {tool_result_content}")

LLM requested a tool call:

- Tool: add_numbers, Args: {"a": 77, "b": 11}

Final Result: 88

weave: 🍩 https://wandb.ai/milieu/london-workshop-2025/r/call/0199ed0d-55df-7b1c-86c1-223969930be9


We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [118]:
messages.append(response.model_dump())

In [119]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": str(tool_result_content)
})

In [120]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant use tools to answer questions.'},
 {'role': 'user',
  'content': 'My lucky numbers are 77 and 11. What is their sum?'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'chatcmpl-tool-b9a99083698b4085a853f8d9748d6ef2',
    'function': {'arguments': '{"a": 77, "b": 11}', 'name': 'add_numbers'},
    'type': 'function'}],
  'reasoning_content': None},
 {'tool_call_id': 'chatcmpl-tool-b9a99083698b4085a853f8d9748d6ef2',
  'role': 'tool',
  'name': 'add_numbers',
  'content': '88'}]

You should have a sequence of messages like this:


In [121]:
[m["role"] for m in messages]

['system', 'user', 'assistant', 'tool']

Now call the model again with the new messages and it will use the tool call result to answer the question

In [122]:
final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.print(final_response.content)

The sum of your lucky numbers, 77 and 11, is 88.

## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [123]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a tool-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": inspect.getdoc(func).split("\\n")[0] if inspect.getdoc(func) else "",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    docstring = inspect.getdoc(func)
    param_descriptions = {}
    if docstring:
        args_section = False
        current_param = None
        for line in docstring.split('\\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped:
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" "): # Heuristic: end of args section
                     args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle List[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by List or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: OpenAI schema doesn't officially use default, but useful metadata

        schema["function"]["parameters"]["properties"][name] = param_schema
        if is_required:
            schema["function"]["parameters"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [124]:
tool_schema = generate_tool_schema(add_numbers)
console.print(tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [125]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        console.print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [126]:
add_numbers = function_tool(add_numbers)
console.print(add_numbers.tool_schema)


{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

In [127]:
add_numbers.tool_schema

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Adds two numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': ''},
    'b': {'type': 'integer', 'description': ''}},
   'required': ['a', 'b']}}}

and call the tool =)

In [128]:
add_numbers(1, 2)

3

### 3.1 Real Example using an API based tool

We are going to use the [EXA search API](https://docs.exa.ai/reference/getting-started).
- How does [EXA search works](https://docs.exa.ai/reference/how-exa-search-works#how-exa-search-works)
- Using exa search [as tool calling](https://docs.exa.ai/reference/openai-tool-calling)

In [129]:
query = "Recipes for cooking seabass?"

search_res = exa_client.search_and_contents(query=query, type='auto', num_results=4)

In [130]:
console.print(search_res)

SearchResponse(
    results=[
        Result(
            url='https://www.masterclass.com/articles/how-to-cook-sea-bass',
            id='https://www.masterclass.com/articles/how-to-cook-sea-bass',
            title='How to Cook Sea Bass: Simple Mediterranean Sea Bass Recipe - 2025 - MasterClass',
            score=None,
            published_date='2025-03-05T00:00:00.000Z',
            author='',
            image='',
            favicon=None,
            subpages=None,
            extras=None,
            text='Is MasterClass right for me?\n\nTake this quiz to find out.\n\nSign 
Up\n\n[All](https://www.masterclass.com/articles/categories/all-classes)[Community and 
Government](https://www.masterclass.com/articles/categories/community-and-government)[Wellness](https://www.masterc
lass.com/articles/categories/wellness)[Design & 
Style](https://www.masterclass.com/articles/categories/design-style)[Arts & 
Entertainment](https://www.masterclass.com/articles/categories/arts-entertainment)[Writing](https://www.masterclass
.com/articles/categories/writing)[Sports & 
Gaming](https://www.masterclass.com/articles/categories/sports-gaming)[Science & 
Tech](https://www.masterclass.com/articles/categories/science-tech)[Music](https://www.masterclass.com/articles/cat
egories/music)[Food](https://www.masterclass.com/articles/categories/food)[Home & 
Lifestyle](https://www.masterclass.com/articles/categories/home-lifestyle)[Business](https://www.masterclass.com/ar
ticles/categories/business)\n\n[Food](https://www.masterclass.com/articles/categories/food)\n\n# How to Cook Sea 
Bass: Simple Mediterranean Sea Bass Recipe\n\nWritten by MasterClass\n\nLast updated: Mar 5, 2025 • 1 min 
read\n\nMediterranean sea bass is an easy-to-cook fish that’s simple enough for a weeknight and luxurious enough 
for a special occasion.\n\n## Learn From the Best\n\nTeaches Gardening\n\nTeaches Intentional Eating\n\nThe 
Greatest of All Time\n\nGut Health with Leading Experts\n\nBrain Health with Leading Experts\n\nTeaches Cooking 
I\n\nTeaches Cooking\n\nTeaches the Art of Home Cooking\n\nTeaches Cooking Techniques I: Vegetables, Pasta, and 
Eggs\n\nTeaches Cooking II: Restaurant Recipes at Home\n\nTeaches Cooking Techniques II: Meats, Stocks, and 
Sauces\n\nTeaches French Pastry Fundamentals\n\nTeaches Wine Appreciation\n\nTeaches Texas-Style BBQ\n\nTeaches 
Modern Italian Cooking\n\nTeaches Cooking Techniques III: Seafood, Sous Vide, and Desserts\n\nTeaches Mexican 
Cooking\n\nTeach Mixology\n\nTeaches Modern Middle Eastern Cooking\n\nTeaches Bread Baking\n\nTeaches Modern 
Japanese Cooking\n\nTeaches Southern Cooking\n\nTeaches Indian Cooking\n\nTeaches Intuitive Cooking\n\nTeaches 
Tracing Your Roots Through Food\n\nTeaches Modern Vegetarian Cooking\n\nSohla El-Waylly with Special Guest 
Chefs\n\nJosé Andrés with Special Guest Chefs\n\nWith Emily Wines\n\nTeaches Gardening\n\nTeaches Intentional 
Eating\n\nThe Greatest of All Time\n\nGut Health with Leading Experts\n\nBrain Health with Leading 
Experts\n\nTeaches Cooking I\n\nTeaches Cooking\n\nTeaches the Art of Home Cooking\n\nTeaches Cooking Techniques I:
Vegetables, Pasta, and Eggs\n\nTeaches Cooking II: Restaurant Recipes at Home\n\nTeaches Cooking Techniques II: 
Meats, Stocks, and Sauces\n\nTeaches French Pastry Fundamentals\n\nTeaches Wine Appreciation\n\nTeaches Texas-Style
BBQ\n\nTeaches Modern Italian Cooking\n\nTeaches Cooking Techniques III: Seafood, Sous Vide, and 
Desserts\n\nTeaches Mexican Cooking\n\nTeach Mixology\n\nTeaches Modern Middle Eastern Cooking\n\nTeaches Bread 
Baking\n\nTeaches Modern Japanese Cooking\n\nTeaches Southern Cooking\n\nTeaches Indian Cooking\n\nTeaches 
Intuitive Cooking\n\nTeaches Tracing Your Roots Through Food\n\nTeaches Modern Vegetarian Cooking\n\nSohla 
El-Waylly with Special Guest Chefs\n\nJosé Andrés with Special Guest Chefs\n\nWith Emily 
Wines\n\n[Get](https://www.masterclass.com/find-my-classes)\n\n## What to Serve With Mediterranean Sea Bass\n\nLike
mo

Let's explore the payload

In [131]:
md("\n-------------------\n".join(result.text for result in search_res.results))

Is MasterClass right for me?                                                                                       

Take this quiz to find out.                                                                                        

Sign Up                                                                                                            

]8;id=833931;https://www.masterclass.com/articles/categories/all-classes\All]8;;\]8;id=787851;https://www.masterclass.com/articles/categories/community-and-government\Community and Government]8;;\]8;id=296460;https://www.masterclass.com/articles/categories/wellness\Wellness]8;;\]8;id=709320;https://www.masterclass.com/articles/categories/design-style\Design & Style]8;;\]8;id=769845;https://www.masterclass.com/articles/categories/arts-entertainment\Arts & Entertainment]8;;\]8;id=830216;https://www.masterclass.com/articles/categories/writing\Writing]8;;\]8;id=441717;https://www.masterclass.com/articles/categories/sports-gaming\Sports & Gaming]8;;\]8;id=486891;https://www.masterclass.com/articles/categories/science-tech\Science & ]8;;\              
]8;id=486891;https://www.masterclass.com/articles/categories/science-tech\Tech]8;;\]8;id=696165;https://www.masterclass.com/articles/categories/music\Music]8;;\]8;id=42569;https://www.masterclass.com/articles/categories/food\Food]8;;\]8;id=743512;https://www.masterclass.com/articles/categories/home-lifestyle\Home & Lifestyle]8;;\]8;id=224210;https://www.masterclass.com/articles/categories/business\Business]8;;\                                                                              

]8;id=107114;https://www.masterclass.com/articles/categories/food\Food]8;;\                                                                                                               

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                           How to Cook Sea Bass: Simple Mediterranean Sea Bass Recipe                            ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Written by MasterClass                                                                                             

Last updated: Mar 5, 2025 • 1 min read                                                                             

Mediterranean sea bass is an easy-to-cook fish that’s simple enough for a weeknight and luxurious enough for a     
special occasion.                                                                                                  


                                                Learn From the Best                                                

Teaches Gardening                                                                                                  

Teaches Intentional Eating                                                                                         

The Greatest of All Time                                                                                           

Gut Health with Leading Experts                                                                                    

Brain Health with Leading Experts                                                                                  

Teaches Cooking I                                                                                                  

Teaches Cooking                                                                                                    

Teaches the Art of Home Cooking                                                                                    

Teaches Cooking Techniques I: Vegetables, Pasta, and Eggs                                                          

Teaches Cooking II: Restaurant Recipes at Home                                                                     

Teaches Cooking Techniques II: Meats, Stocks, and 

In [132]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.
    
    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.
    
    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.
    
    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)
    
    output = []
    for result in search_results.results:
        output.append(
            {"title": result.title,
            "text": result.text,
            "url": result.url
            }
        )
    return output

    

In [133]:
exa_search.tool_schema

{'type': 'function',
 'function': {'name': 'exa_search',
  'description': 'Perform a search query on the web and retrieve the most relevant URLs and web content.\n\nThis function uses the Exa search API to find relevant web pages based on the query\nand returns their titles, text content, and URLs.\n\nArgs:\n    query: The search query. Use detailed, specific queries for better results.\n           The quality of results depends on the specificity of the query.\n    num_results: The number of search results to retrieve. Defaults to 5.\n\nReturns:\n    A list of dictionaries, each containing:\n        - title: The title of the web page\n        - text: The text content of the web page\n        - url: The URL of the web page',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string', 'description': ''},
    'num_results': {'type': 'integer', 'description': '', 'default': 5}},
   'required': ['query']}}}

We get a list of results with the relevant metadata.

In [134]:
search_results = exa_search("How do I cook seabass?")
console.print(search_results)

[
    {
        'title': 'How to cook seabass',
        'text': '# Seabass\n\nSince its debut on the UK restaurant scene in the 1980s, sea bass has become a 
superstar fish, but at what cost, asks Clarissa Hyman?\n\nI am always a little puzzled by the description of 
_Dicentrarchus labrax_ as sea bass. Not that I insist on ordering my main course in Latin, although it would be\na 
conversational opener (or relationship\nstopper), but because it is axiomatic. I\nmean, where else is the fish to 
come from?\nThe swimming pool? The bathtub? Of\ncourse it’s a fish from the sea, as opposed\nto a freshwater fish, 
but we don’t go round\nsaying sea cod or sea haddock, do we?\n\nPrized since Roman times, bass has\nbeen described 
as a handsome and\nvoracious fish, which makes me think of it\nas something of a deep-sea player, a kind\nof 
keen-eyed Sean Penn with silvery flanks\nand chain-mail scales. It’s a favourite with\nanglers, especially in the 
south of Ireland,\nwhere it is sometimes called ‘bairs’, a word\nmore similar to the Danish ‘bars’ and the 
Anglo-Saxon ‘baers’ rather than the English\n‘bass’. Why that should be, I have no idea.\nThey also used to be 
called ‘salmon\nbass’ by fishwives in the west of England\nbecause of a superficial similarity in looks,\nalthough 
they are in no way connected\nwith the salmon family. They are found as\nfar south as Senegal, through 
the\nMediterranean and in the East Atlantic up\nas far as Norway. Their most famous\ncousin is the American striped
bass or\n‘striper’ – easily identified by the parallel,\ngo-faster black stripes along the platinum\nbody – which 
ranges from the Gulf of\nSt Lawrence down to the Gulf of Mexico.\nIt was successfully introduced to the\nPacific 
coast in the 19th century and now is an important sporting game fish.\nBass has become a top restaurant\nfavourite 
as it boasts firm, delicate but\nwell-flavoured flesh, is relatively free of\nsmall bones and holds its shape well 
after\ncooking. They look particularly fine when\npresented whole, usually roasted, as they\nrarely grow longer 
than 80cm, but they can\nbe cooked in a great variety of ways –\nsteamed, boiled, grilled, fried, poached,\nstewed 
or baked. The simplest method,\nhowever, as experience regularly teaches\nus, is often the best. One excellent 
and\nmouthwatering way is to bake it Santander\nstyle, strewn with a mixture of toasted\nalmonds, toasted 
breadcrumbs and parsley.\nBass pairs particularly well with other\nMediterranean flavours: tomatoes, garlic,\nolive
oil and red peppers, as well as\noriental flavours, such as fermented black\nbeans, sesame, lemongrass, dark soy 
and\nginger. A whole fish baked in salt is a\nclassic Italian dish; cooking bass _en_\n_papillote_ with aromatic 
flavours is also\ntypical. Rather like cod, the cheeks of bass\nare sweet and lightly flavoured, and 
are\nconsidered a delicacy. The skin too, when crispy, is excellent eating.\nThey are said to be a remarkably 
intelligent fish in so far as they seem to be able to spot a fishing\nboat when conditions are clear and 
scarper\nas fast as they can. Conversely rough seas\nand poor visibility for the fish mean a better\ncatch. Their 
popularity, however, has led to\noverfishing of wild stocks: their vulnerability\nhas been increased by a toxic 
combination\nof illegal fishing, slow growth, late maturity,\nspawning aggregation and strong summer-\nsite 
preference leading to exploitation and\nlocalised depletion in certain areas.\n\nIn 2017, the Marine Conservation 
Society\n(MCS) announced the spawning biomass\nfor the stock around the British and Irish\ncoasts were at their 
lowest observed level\nand scientists from the International Council\nfor the Exploration of the Sea (ICES) 
advised\nthere should be zero catch (commercial\nand recreational). Various fisheries in the Northeast Atlantic 
such as that in the Bristol\nChannel, and even the Dutch rod and line\nfishery, have had their Marine 
Stewardship\nCouncil (MSC)

In [135]:
messages = [
    {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
    {"role": "user", "content": "How do I cook seabass?"}]

response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[exa_search.tool_schema])
console.print(response)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-e94c0d96dd0a46a9af31b494dbd81571',
            function=Function(
                arguments='{"query": "how to cook seabass step by step", "num_results": 5}',
                name='exa_search'
            ),
            type='function'
        )
    ],
    reasoning_content=None
)

Let's create some helper functions to perform the tool calls

In [140]:
from openai.types.chat.chat_completion_message_function_tool_call import ChatCompletionMessageFunctionToolCall

def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], tool_calls: list[ChatCompletionMessageFunctionToolCall]) -> list[dict]:
    "Perform the tool calls and return the messages with the tool call results"
    messages = []
    for tool_call in tool_calls:
        console.rule(f"Performing tool call: {tool_call.function.name}")
        console.print(f"  - Args: {tool_call.function.arguments}")
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        tool = get_tool(tools, function_name)
        tool_response = tool(**function_args)
        console.print(f"  - Context Updated with: {tool_response}")
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "content": str(tool_response),
        })
    return messages

In [141]:

# add the tool call result to the messages
messages.append(response.model_dump())
messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
messages.append({
    "role": "user",
    "content": "Answer my previous query based on the search results.",})

final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.rule("Final Model Response")
md(final_response.content)


──────────────────────────────────────── Performing tool call: exa_search ─────────────────────────────────────────

- Args: {"query": "how to cook seabass step by step", "num_results": 5}

- Context Updated with: [{'title': "Chilean Sea Bass Recipe {Restaurant Style} - The Big Man's World ®", 'text': 
"[Skip to content](https://thebigmansworld.com/thebigmansworld.com#main-content)\n\n# Chilean Sea Bass Recipe\n\nBy
[Arman Liew](https://thebigmansworld.com/arman-liew/)published on May 20, 2025\n\n[Jump to Recipe 
▼](https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)\n\nReader Rating\n\nTotal Time
10 minutesmins\n\nServings 4servings\n\n_This post may contain affiliate links. See my 
(https://thebigmansworld.com/privacy-policy/)._\n\n_This pan seared **Chilean sea bass recipe** comes together in 
under 10 minutes and cooks perfectly every single time! It’s easy, elegant, and delicious._\n\n**★★★★★ 
REVIEW**\n\n_“OMG, this recipe was the easiest and the most delicious! My sister had never had sea bass before, so 
I made it for both of us. I did get it without the skin, and it was absolutely amazing! If you haven’t tried this, 
please use this recipe, you won’t regret it, the absolute best!”_ – Pam\n\nTable of Contents\n\n1. [Key 
Ingredients](https://thebigmansworld.com/thebigmansworld.com#h-key-ingredients)\n2. [How to cook Chilean sea 
bass](https://thebigmansworld.com/thebigmansworld.com#h-how-to-cook-chilean-sea-bass)\n3. [Chilean sea bass cooking
tips](https://thebigmansworld.com/thebigmansworld.com#h-chilean-sea-bass-cooking-tips)\n4. [How to 
serve](https://thebigmansworld.com/thebigmansworld.com#h-how-to-serve)\n5. [Seared Chilean Sea Bass Recipe (Recipe 
Card)](https://thebigmansworld.com/thebigmansworld.com#wprm-recipe-container-61837)\n\nFriends, if you are looking 
for a simple fish dinner that’s elegant enough for date night, a dinner party, or a special occasion, my quick, 
pan-seared Chilean sea bass recipe is a must-try. While it’s a popular choice at fine dining restaurants or elegant
events, making it at home is SO easy and tastes just as good- if not better!\n\nIt’s so tender, flaky, and 
naturally buttery that the sea bass practically melts in your mouth. I love that it takes just ten minutes to make 
(prep AND cook time included) and has minimal clean-up. It’s drizzled with a simple garlic lemon butter, which lets
the natural flavor of the Chilean sea bass shine.\n\nIf you love elegant seafood dinners, try my 
**(https://thebigmansworld.com/parmesan-crusted-tilapia/)**, **(https://thebigmansworld.com/grilled-tuna-steak/)**,
**(https://thebigmansworld.com/branzino-recipe/)**, and 
**(https://thebigmansworld.com/pan-seared-rockfish-recipe/)** next.\n\nWhat is Chilean Sea Bass?\n\nChilean sea 
bass (also called Patagonian toothfish, mero, or icefish), is a deep-water species native to the cold sub-Antarctic
regions of the Pacific, southern Atlantic, and Indian Oceans.\n\nIt’s high in protein and super low in carbs. It’s 
famous for its silky, buttery texture and mild flavor (no fishy taste). It’s considered a premium fish due as its 
hard to overcook and super versatile.\n\n## Key Ingredients\n\n- **Chilean sea bass fillets**. Either fresh or 
frozen fillets.\xa0Because of the slightly higher price tag, I prefer to get the fillets from a fishmonger over a 
grocery store. When choosing sea bass, opt for fillets that have no strong odors, no dry spots, and are firm to 
touch.\n- **Kosher salt and black pepper**. To taste.\n- **Unsalted butter**. Pan-frying white fish fillets in 
butter takes the dish’s flavor to another level and cooks the fish perfectly. Olive oil will also work.\n- 
**Garlic**. Freshly minced garlic, not the jarred kind!\n- **Lemon**. Fresh lemons to squeeze over the freshly 
cooked fish.\n\n## How to cook Chilean sea bass\n\n_This is an overview with step-by-step photos. Full ingredients 
& instructions are in the 
(https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)._\n\n**Step 1-** Pat dry the 
fish filets with a paper towel.\n\n**Step 2-** Sprinkle one side of each filet with salt and pepper.\n\n**Step 3-**
Melt butt

Generating final Response:

────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Cooking sea bass can be done in several delicious ways, with pan-searing and baking being the most popular methods.
Here’s a comprehensive guide based on the search results:                                                          

                                     Pan-Seared Sea Bass (Most Common Method)                                      

This method results in a crispy exterior and tender, flaky interior.                                               

 1 Prepare the Fish:                                                                                               
    • Pat the sea bass fillets completely dry with paper towels. This is crucial for achieving a good sear and     
      preventing sticking.                                                                                         
    • Season both sides generously with salt and pepper. You can also use other seasonings like garlic powder,     
      paprika, cumin, or coriander.                                                                                
 2 Sear the Fish:                                                                                                  
    • Heat a skillet (preferably cast iron or non-stick) over medium-high heat. Add a high smoke-point oil (like   
      avocado oil) or a combination of oil and butter.                                                             
    • Once the pan is very hot, place the fillets skin-side down (if they have skin). This helps create a crispy   
      skin and keeps the fish moist.                                                                               
    • Do not move the fish for the first few minutes. Let it sear undisturbed for 2-5 minutes, depending on the    
      thickness, until the skin is golden brown and releases easily from the pan.                                  
 3 Finish Cooking:                                                                                                 
    • Carefully flip the fillets using a fish spatula.                                                             
    • Cook for another 2-4 minutes on the other side until the fish is opaque and flakes easily with a fork.       
    • Check for doneness: The internal temperature should reach 140-145°F (60-63°C).                               
 4 Serve:                                                                                                          
    • Remove the fish from the pan and immediately drizzle with lemon juice.                                       
    • You can also finish it with a sauce, such as a simple lemon-butter sauce, a lemon-caper butter, or a         
      garlic-lemon butter.                                                                                         

                                                  Baked Sea Bass                                                   

A simple, low-mess method that keeps the fish moist.                                                               

 1 Preheat & Prepare:                                                                                              
    • Preheat your oven to 400°F (200°C).                                                                          
    • Let the fish come to room temperature for about 20 minutes for even cooking.                                 
    • Pat the fillets dry and brush them with melted butter or olive oil.                                          
    • Season generously with salt, pepper, and optional paprika.                                                   
 2 Bake:                                                                                                           
    • Place the fillets on a parchment-lined baking dish.                                                          
    • Bake for 12-15 minutes, or until the fish is opaque and flakes easily. The exact time depends on the         
      thickness of the fillets.                   

Let's wrap this in a function and add a few more tools.

In [142]:
@weave.op
def research(query: str) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant that can use tools to answer questions."},
        {"role": "user", "content": query}]

    # call model with tools
    response = call_model(
        model_name=MODEL_SMALL, 
        messages=messages, 
        tools=[exa_search.tool_schema])

    # add the response to the messages
    messages.append(response.model_dump())

    # perform the tool calls
    messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
    
    # prompt the model to be grounded
    messages.append({"role": "user","content": "Answer my previous query based on the search results.",})

    final_response = call_model(model_name=MODEL_SMALL, messages=messages)
    return final_response.content

In [143]:
result = research("What are the most popular pokemons?")
md(result)

──────────────────────────────────────── Performing tool call: exa_search ─────────────────────────────────────────

- Args: {"query": "most popular Pokémon of all time", "num_results": 5}

- Context Updated with: [{'title': 'Most Popular Pokémon 2004 - 2020 (Official Data) - YouTube', 'text': '► Most 
Popular Pokémon 2004 - 2020 : Eeveelutions, Legendary, Starters, Fire, Water, Grass - 
•\xa0Most\xa0Popular\xa0Pokémon\xa02004\xa0-\xa02020\xa0:\xa0Eeveelu... \\nTimeline of the most searched Pokemon on
the Internet from 2004 to 2020. Worldwide statistics based on websites traffic measured by monthly visits. Who is 
your favourite Pokemon and did they make it on the list?\n| view_count: 4,526,692 views | short_view_count: 4.5M 
views | num_likes: 88 thousand | num_subscribers: 2.75 million', 'url': 
'https://www.youtube.com/watch?v=b_RLZiP49PU'}, {'title': '25 Most Popular Pokémon From the Original 151, Ranked - 
Collider', 'text': 'Close\n\nClose\n\nThis article will refer to the English-translated versions of characters and 
events. Pokémon has grown to be one of the most iconic and highest-grossing media franchises of all time, between 
the video games, trading card games, animated series, and feature films, there is no shortage of exceptional 
Pokémon content out there. There are now over a thousand different Pokémon creatures that make up the vast and 
expansive world of Pokémon, yet there is still something special about the original set of 151 Pokémon that has 
fans enamored 25 years later.\n\n**Being the first set of Pokémon released, many of these creatures have become 
icons** not just of the humble beginnings of the franchise, but icons for the entire series as a whole, even going 
beyond the popularity of Pokémon itself. Popularity is far from a defining factor when it comes to favoritism, as 
every individual is going to have their favorite Pokémon, it just so happens that these frequently find themselves 
as favorites over their contemporaries.\n\n## 25 Ninetales\n\n### Pokédex No. \\#038\n\nThe fire type fox pokemon 
Ninetales sitting on top of a rock with clouds in the background in the Pokemon anime series.\n\nOne of the most 
gracious and awe-inspiring fire-type Pokémon of all time, the inherent allure and majesty of Ninetales have 
continued to make it an icon well after its debut in the original 151. Evolving from the incredibly cute Vulpix, 
Ninetales\' titular bunch of tales and glowing blonde fur give it **a sort of mythical, enchanting appearance as if
it hopped straight out of the pages of a children\'s fairy tale**. Its memorability has also been continued by an 
equally beautiful alternate form, with Alolan Ninetales swapping its fire type for an Ice and Fairy type.\n\nIn the
animated series, Ninetales is most commonly remembered as a part of gym leader Blaine\'s team during his battle 
against Ash, battling off against Squirtle and winning the losing type-matchup. However, both forms of the Pokémon 
continue to be utilized throughout the series, such as an ancient Ninetales befriending Brock as well as Goh 
catching an Alolan Ninetales in _**Pokémon Journeys: The Series**_.\n\n## 24 Lapras\n\n### Pokédex No. 
\\#131\n\nEasily one of the most elegant and majestic Pokémon from the original 151, there is an allure of pristine
beauty and grace that is present throughout Lapras, the Water/Ice-type Pokémon synonymous with the act of traveling
across bodies of water. The single-evolution Pokémon has been a fan-favorite water type ever since its 
introduction, especially since its prospects as a gift Pokémon in the original game meant that it wasn\'t hard at 
all to obtain the beautiful creature.\n\nLapras has also made frequent appearances in the anime over the years, 
with Ash himself saving a Lapras during his time venturing across the Orange Islands, using Lapras as transport 
between the various islands. Even outside of Ash, Lapras consistently finds itself recurring throughout the 
widespread Pokémon series and anime, **often as the quintessential staple and symbol of sea transport in the 
franchise**. Most recently, Lapras has been seen to be a Pokémon owned by Lucius in _**Pokémon Horizo

Based on the search results, the most popular Pokémon vary depending on the context—such as fan surveys, trading   
card game (TCG) value, or cultural recognition—but several names consistently stand out.                           

According to a large fan survey of over 52,000 Reddit users conducted in 2019 (as reported by Business Insider),   
the top 5 most beloved Pokémon were:                                                                               

 1 Charizard – Fire/Flying type, known for its power and iconic status.                                            
 2 Gengar – Ghost/Poison type, beloved for its design and personality.                                             
 3 Arcanine – Fire type, praised for its majestic appearance and loyalty.                                          
 4 Bulbasaur – Grass/Poison type, surprisingly topping Pikachu in this poll.                                       
 5 Blaziken – Fire/Fighting type, a fan-favorite starter from Generation III.                                      

Interestingly, Pikachu, the franchise mascot, ranked only 44th in this particular fan vote, though it remains the  
most recognizable Pokémon globally.                                                                                

Other popular Pokémon frequently mentioned across sources include:                                                 

 • Eevee and its evolutions (especially Vaporeon, Jolteon, Umbreon)                                                
 • Dragonite                                                                                                       
 • Lucario                                                                                                         
 • Gyarados                                                                                                        
 • Mewtwo and Mew (as legendary favorites)                                                                         
 • Rayquaza                                                                                                        

In the Pokémon TCG (Trading Card Game), popularity is often reflected in card prices and demand, with Charizard,   
Pikachu, Umbreon, Gengar, and Rayquaza being especially valuable and sought-after.                                 

In summary, while Charizard emerges as the most popular Pokémon in fan-driven rankings, overall popularity spans a 
mix of starters, legendaries, and fan-designed favorites, with Pikachu remaining the most iconic face of the       
franchise.

![](images/04_pokedex.png)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](images/05_agent.png)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [73]:
@weave.op
def pokedex_loop(pokemon_question: str, max_turns: int = 4, tools = [get_pokemon_info, add_numbers]) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant that can use tools to answer questions."},
        {"role": "user", "content": pokemon_question}]

    
    for turn in range(max_turns):
        print(f"--- Agent Loop Turn {turn + 1}/{max_turns} ---")

        # call model with tools
        response = call_model(
            model_name=MISTRAL_SMALL_MODEL, 
            messages=messages, 
            tools=[t.tool_schema for t in tools])

        # add the response to the messages
        messages.append(response.model_dump())

        # if the LLM requested tool calls, perform them
        if response.tool_calls:
            print("LLM requested tool calls:")
            # perform the tool calls
            tool_outputs = perform_tool_calls(tools=[get_pokemon_info, add_numbers], tool_calls=response.tool_calls)
            messages.extend(tool_outputs)
        # LLM gave content response
        elif response.content:
            print(f"LLM content response: {response.content}")
            return response.content
        else:
            print("LLM response had neither content nor tool calls. Stopping loop.")
            break

In [74]:
pokedex_loop("What is the combined weight of Ash's first 3 pokemons?")

--- Agent Loop Turn 1/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "pikachu"}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "caterpie"}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "pidgey"}
  - Response: {'name': 'Pidgey', 'id': 16, 'weight': 18}
--- Agent Loop Turn 2/4 ---
LLM requested tool calls:
Performing tool call: add_numbers
  - Args: {"a": 60, "b": 29}
  - Response: 89
Performing tool call: add_numbers
  - Args: {"a": 60, "b": 18}
  - Response: 78
--- Agent Loop Turn 3/4 ---
LLM content response: The combined weight of Ash's first 3 pokemons is 167 hectograms.


"The combined weight of Ash's first 3 pokemons is 167 hectograms."

In [75]:
pokedex_loop("Name Ash's first 3 pokemons and calculate their combined weight?")

--- Agent Loop Turn 1/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "pikachu"}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "caterpie"}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "pidgey"}
  - Response: {'name': 'Pidgey', 'id': 16, 'weight': 18}
--- Agent Loop Turn 2/4 ---
LLM content response: Ash's first three Pokémon are Pikachu, Caterpie, and Pidgey. Their combined weight is 107 hectograms.


"Ash's first three Pokémon are Pikachu, Caterpie, and Pidgey. Their combined weight is 107 hectograms."

![](images/06_pokedex_loop.png)

This one is kind of an hallucination, I was expecting snorlax to be the heaviest pokemon.

In [76]:
pokedex_loop("Which is the heaviest pokemon?")

--- Agent Loop Turn 1/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "1"}
  - Response: {'name': 'Bulbasaur', 'id': 1, 'weight': 69}
--- Agent Loop Turn 2/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "2"}
  - Response: {'name': 'Ivysaur', 'id': 2, 'weight': 130}
--- Agent Loop Turn 3/4 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "3"}
  - Response: {'name': 'Venusaur', 'id': 3, 'weight': 1000}
--- Agent Loop Turn 4/4 ---
LLM content response: The heaviest pokemon is Venusaur, with a weight of 1000 hectograms.


'The heaviest pokemon is Venusaur, with a weight of 1000 hectograms.'

6. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [77]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: List[Dict[str, Any]] = Field(default_factory=list)
    step: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

In [82]:
class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: List[Callable]):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
    
    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step = state.step + 1
        messages = state.messages
        final_assistant_content = None
        try:
            # call model with tools
            response = call_model(
                model_name=self.model_name, 
                messages=messages, 
                tools=[t.tool_schema for t in self.tools])

            # add the response to the messages
            messages.append(response.model_dump())

            # if the LLM requested tool calls, perform them
            if response.tool_calls:
                print("LLM requested tool calls:")
                # perform the tool calls
                tool_outputs = perform_tool_calls(tools=[get_pokemon_info, add_numbers], tool_calls=response.tool_calls)
                messages.extend(tool_outputs)

            # LLM gave content response
            else:
                messages.append(response.model_dump())
                final_assistant_content = response.content
        except Exception as e:
            print(f"ERROR in Agent Step: {e}")
            # Add an error message to history to indicate failure
            messages.append({"role": "assistant", "content": f"Agent error in step: {str(e)}"})
            final_assistant_content = f"Agent error in step {step}: {str(e)}"
        return AgentState(messages=messages, step=step, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10) -> AgentState:
        state = AgentState(messages=[
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": user_prompt}])
        for _ in range(max_turns):
            print(f"--- Agent Loop Turn {state.step}/{max_turns} ---")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state


![](images/07_simple_agent.png)

In [ ]:
agent = SimpleAgent(
    model_name=MISTRAL_SMALL_MODEL,
    system_message="You are a helpful assistant that can use tools to answer questions.",
    tools=[get_pokemon_info, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")


--- Agent Loop Turn 0/10 ---
LLM requested tool calls:
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "Pikachu"}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "Caterpie"}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29}
Performing tool call: get_pokemon_info
  - Args: {"pokemon_name": "Pidgey"}
  - Response: {'name': 'Pidgey', 'id': 16, 'weight': 18}
--- Agent Loop Turn 1/10 ---
LLM requested tool calls:
Performing tool call: add_numbers
  - Args: {"a": 60, "b": 29}
  - Response: 89
Performing tool call: add_numbers
  - Args: {"a": 60, "b": 18}
  - Response: 78
--- Agent Loop Turn 2/10 ---
The combined weight of Ash's first 3 pokemons is 89 + 78 = 167 hectograms.


Possible improvements to the SimpleAgent:
- Give the model info about the state of the conversation, you could inject a message with the model context pressure, steps left, etc.
- Structured output. Make the model output a specific format, for instance a JSON with the expected fields.
- Add more tools like read and write files, access a database.
- Agent handoff: Agent1 does triage and Agent2 executes specific tasks.